In [67]:
import pandas as pd
import re
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [68]:
browser = webdriver.Firefox()
browser.get('https://www.tripadvisor.fr')
time.sleep(2)
browser.maximize_window()

# Recherche : restaurants sur Paris

In [69]:
restaurant_btn = browser.find_element(By.CSS_SELECTOR, 'a[data-automation="centralNav_restaurants"]')
restaurant_btn.click()
time.sleep(2)

search_bar = browser.find_element(By.CSS_SELECTOR, 'input[title="Rechercher"]')
search_bar.send_keys('Paris')
time.sleep(3)

browser.find_element(By.ID, 'typeahead_results').find_element(By.TAG_NAME, 'a').click()

# Clique sur un élément

In [70]:
# TODO: modifier pour utiliser une classe ?
def get_restaurant_page_info(browser):
    """Prend en paramètre un WebDriver et récupère le nom, les catégories et l'adresse sur la page d'un restaurant et les renvoie dans un dictionnaire.

    Exemple d'utilisation : `get_restaurant_page_info(browser)`

    Exemple de résultat :
    ```json
    {
      'nom': 'Le Jules Verne',
      'categorie': ['Française', 'Européenne'],
      'adresse': 'Avenue Gustave Eiffel, 75007 Paris France'
    }
    ```
    """
    res_info_div = browser.find_element(By.CSS_SELECTOR, 'div[data-test-target="restaurant-detail-info"]')

    categories = res_info_div.find_elements(By.CLASS_NAME, 'dlMOJ')

    return {
        'nom': res_info_div.find_element(By.TAG_NAME, 'h1').text,
        'categorie': [cat.text for cat in categories if '€' not in cat.text],
        'adresse': res_info_div.find_elements(By.CLASS_NAME, 'AYHFM')[1].text,
    }

In [71]:
def switch_to_next_page(browser):
    """Prend en paramètre un WebDriver et clique sur le bouton \"Suivant\" s'il existe.
    
    Exemple d'utilisation : `switch_to_next_page(browser)`
    """
    try:
        next_page_btn = browser.find_element(By.CSS_SELECTOR, 'a[data-smoke-attr="pagination-next-arrow"]')
        
        if next_page_btn:
            browser.execute_script(f'window.scrollTo(0, {next_page_btn.rect["y"] - 300})')

            next_page_btn.click()
    except Exception as e:
        print('Erreur lors de la tentative de changement de page :', e)
        pass

In [72]:
index = 0
page = 1

restaurants = []

def get_all_restaurant_divs(browser):
    """Prend en paramètre un WebDriver et renvoie une liste de tous les restaurants (div en HTML).
    
    Exemple d'utilisation : `get_all_restaurant_divs(browser)`
    """
    elements = browser.find_element(By.CSS_SELECTOR, 'div[class="Ikpld f e"]').find_elements(By.CSS_SELECTOR, 'div[data-test]')
    return [el for el in elements if re.search(r'\d+_list_item', el.get_attribute('data-test'))]

while page < 4:
    print('URL de la page :', browser.current_url)

    # Rafraîchir la liste des restaurants après chaque changement de page
    restaurants_html = get_all_restaurant_divs(browser)

    print('Nombre de restaurants trouvés sur cette page :', len(restaurants_html))

    while index < len(restaurants_html):
        try:
            res = restaurants_html[index]
            lien_avis = res.find_element(By.TAG_NAME, 'a')

            browser.execute_script(f'window.scrollTo(0, {lien_avis.rect["y"] - 300})')
            time.sleep(2)

            browser.get(lien_avis.get_attribute('href'))
            time.sleep(4)

            # Récupération des informations du restaurant
            variable_to_rename = get_restaurant_page_info(browser)
            print(variable_to_rename)
            restaurants.append(variable_to_rename)

            # Retour à la liste des restaurants
            browser.back()
            time.sleep(2)

            index += 1

        # Si la liste de restaurants (div) est obsolète, rafraîchir
        except StaleElementReferenceException:
            restaurants_html = get_all_restaurant_divs(browser)
            continue

    # Si on arrive au bout de la liste des restaurants, on passe à la page suivante
    if index >= len(restaurants_html):
        switch_to_next_page(browser)
        page += 1
        time.sleep(4)
        index = 0

print('nombre de restaurants :', len(restaurants))


URL de la page : https://www.tripadvisor.fr/Restaurants-g187147-Paris_Ile_de_France.html


Nombre de restaurants trouvés sur cette page : 30
{'nom': 'New Jawad Longchamp', 'categorie': ['Indienne', 'Pakistanaise', 'Végétariens bienvenus'], 'adresse': '30 rue de Longchamp, 75116 Paris France'}
{'nom': 'Orgueil', 'categorie': ['Française', 'Saine'], 'adresse': '6 rue Popincourt, 75011 Paris France'}
URL de la page : https://www.tripadvisor.fr/Restaurants-g187147-oa30-Paris_Ile_de_France.html
Nombre de restaurants trouvés sur cette page : 30
{'nom': 'Frog & Underground', 'categorie': ['Américaine', 'Bar à bières', 'Bar'], 'adresse': '176 Rue Montmartre, 75002 Paris France'}
{'nom': 'Pardi', 'categorie': ['Française', 'Méditerranéenne', 'Européenne'], 'adresse': '85 B boulevard de Magenta Marché Saint Quentin, 75010 Paris France'}
nombre de restaurants : 4


In [ ]:
test = browser.find_element(By.CSS_SELECTOR, 'span[class="IiChw"]')
test.click()
time.sleep(3)

In [42]:
# Utiliser un sélecteur XPath pour trouver l'élément
elements = browser.find_element(By.CLASS_NAME, 'class="partial_entry"')

print(elements)

NoSuchElementException: Message: Unable to locate element: .rmyCe _G B- z _S c Wc wSSLS jWkoZ XDcOZ; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16
